<a href="https://colab.research.google.com/github/svecx/Report-Script/blob/staging/Prod_Matrix_Sap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
from datetime import datetime
!pip install XlsxWriter

In [21]:
def load_data():
    df_title = datetime.today().strftime('%d%m%y')
    df = pd.read_excel(f'{df_title}_ ZF1_Prod Matrix SAP.XLSX', keep_default_na=False)

    D_BNLN1 = pd.read_excel('/content/LNPAR2.XLSX', keep_default_na=False)
    D_BNCD = pd.read_excel('/content/CDPAR2.XLSX', keep_default_na=False)
    D_BNDD = pd.read_excel('/content/DDPAR2.XLSX', keep_default_na=False)

    RAW = df
    BNLN = df[df['Source System Basic Data'] == "BNLN"]
    BNCD = df[df['Source System Basic Data'] == "BNCD"]
    BNDD = df[df['Source System Basic Data'] == "BNDD"]

    return df, D_BNLN1, D_BNCD, D_BNDD, RAW, BNLN, BNCD, BNDD

In [22]:
def merge_data(D_BNLN1, D_BNCD, D_BNDD, BNLN, BNCD, BNDD):
    try:
        dfr_D_BNLN1 = pd.merge(
            D_BNLN1,
            BNLN[['Original Source Product Code']].rename(columns={'Original Source Product Code': 'SAP'}),
            left_on="PTYPE",
            right_on="SAP",
            how="left"
        )
        cols = dfr_D_BNLN1.columns.tolist()
        if 'SAP' in cols and 'PTYPE' in cols:
            cols.insert(cols.index('PTYPE') + 1, cols.pop(cols.index('SAP')))
            dfr_D_BNLN1 = dfr_D_BNLN1[cols]
    except Exception as e:
        print(f"❌ Merge D_BNLN1 gagal: {e}")
        dfr_D_BNLN1 = pd.DataFrame()

    try:
        dfr_D_BNCD = pd.merge(
            D_BNCD,
            BNCD[['Original Source Product Code']].rename(columns={'Original Source Product Code': 'SAP'}),
            left_on="PTYPE",
            right_on="SAP",
            how="left"
        )
        cols = dfr_D_BNCD.columns.tolist()
        if 'SAP' in cols and 'PTYPE' in cols:
            cols.insert(cols.index('PTYPE') + 1, cols.pop(cols.index('SAP')))
            dfr_D_BNCD = dfr_D_BNCD[cols]
    except Exception as e:
        print(f"❌ Merge D_BNCD gagal: {e}")
        dfr_D_BNCD = pd.DataFrame()

    try:
        D_BNDD['SCCODE'] = D_BNDD['SCCODE'].str.upper()
        dfr_D_BNDD = pd.merge(
            D_BNDD,
            BNDD[['Original Source Product Code']].rename(columns={'Original Source Product Code': 'SAP'}),
            left_on="SCCODE",
            right_on="SAP",
            how="left"
        )
        cols = dfr_D_BNDD.columns.tolist()
        if 'SAP' in cols and 'SCCODE' in cols:
            cols.insert(cols.index('SCCODE') + 1, cols.pop(cols.index('SAP')))
            dfr_D_BNDD = dfr_D_BNDD[cols]
    except Exception as e:
        print(f"❌ Merge D_BNDD gagal: {e}")
        dfr_D_BNDD = pd.DataFrame()

    return dfr_D_BNLN1, dfr_D_BNCD, dfr_D_BNDD

In [23]:
def validate_data(dfr_D_BNLN1, dfr_D_BNCD, dfr_D_BNDD):
    dfr_D_BNLN1['SAP'] = dfr_D_BNLN1['SAP'].fillna("#NA")
    dfr_D_BNCD['SAP'] = dfr_D_BNCD['SAP'].fillna("#NA")
    dfr_D_BNDD['SAP'] = dfr_D_BNDD['SAP'].fillna("#NA")

    C_dfr_D_BNLN1 = (dfr_D_BNLN1['SAP'] == '#NA').sum()
    C_dfr_D_BNCD = (dfr_D_BNCD['SAP'] == '#NA').sum()
    C_dfr_D_BNDD = (dfr_D_BNDD['SAP'] == '#NA').sum()

    summary_df = pd.DataFrame({
        'Nama': ['D_BNLN1', 'D_BNCD', 'D_BNDD'],
        'Jumlah': [C_dfr_D_BNLN1, C_dfr_D_BNCD, C_dfr_D_BNDD]
    })

    validations = [
        ("D_BNLN1 harus < 1", C_dfr_D_BNLN1 < 1),
        ("D_BNCD harus < 1", C_dfr_D_BNCD < 1),
        ("D_BNDD harus < 1", C_dfr_D_BNDD < 1),
    ]

    invalids = [f"❌ {msg}" for msg, condition in validations if not condition]

    if invalids:
        print("📌 Masalah ditemukan:\n" + "\n".join(invalids))
    else:
        print("✅ Semua kondisi terpenuhi, data aman.")

    return summary_df

In [24]:
def save_to_excel(RAW, BNLN, BNCD, BNDD, dfr_D_BNLN1, dfr_D_BNCD, dfr_D_BNDD):
    today = datetime.today().strftime('%d-%m-%Y')
    filename = f"Merge Prod Matrix Sap {today}.xlsx"

    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        RAW.to_excel(writer, sheet_name="RAW", index=False)
        BNLN.to_excel(writer, sheet_name="BNLN", index=False)
        BNCD.to_excel(writer, sheet_name="BNCD", index=False)
        BNDD.to_excel(writer, sheet_name="BNDD", index=False)
        dfr_D_BNLN1.to_excel(writer, sheet_name="D_BNLN1", index=False)
        dfr_D_BNCD.to_excel(writer, sheet_name="D_BNCD", index=False)
        dfr_D_BNDD.to_excel(writer, sheet_name="D_BNDD", index=False)

    print(f"📂 File berhasil disimpan: {filename}")

In [25]:
def main():
    df, D_BNLN1, D_BNCD, D_BNDD, RAW, BNLN, BNCD, BNDD = load_data()
    dfr_D_BNLN1, dfr_D_BNCD, dfr_D_BNDD = merge_data(D_BNLN1, D_BNCD, D_BNDD, BNLN, BNCD, BNDD)
    summary_df = validate_data(dfr_D_BNLN1, dfr_D_BNCD, dfr_D_BNDD)
    save_to_excel(RAW, BNLN, BNCD, BNDD, dfr_D_BNLN1, dfr_D_BNCD, dfr_D_BNDD)
    return summary_df


# 🚀 Jalankan
main()

✅ Semua kondisi terpenuhi, data aman.
📂 File berhasil disimpan: Merge Prod Matrix Sap 20-08-2025.xlsx


,Nama,Jumlah
0,D_BNLN1,0
1,D_BNCD,0
2,D_BNDD,0


# **Pengecekan Error**
---

